In [60]:
import Pkg
Pkg.activate("..")

# these lines are required only for local installations
# Pkg.develop(path="../../scp_traj_opt/")
# Pkg.precompile()

using SCPToolbox
using COSMO
using ECOS

  Activating project at `~/Nextcloud/personal/research/scp/aas_gnc_workshop/tutorial`


# Problem data

In [61]:
q = [1; 1.];
P = [4. 1; 1 2];
A = [1. 1; 1 0; 0 1];
l = [1.; 0; 0];
u = [1; 0.7; 0.7];

# First, we decide to solve the problem with two one-sided constraints using `COSMO.Nonnegatives` as the convex set:
Aa = [-A; A]
ba = [u; -l]
;

# JuMP

In [62]:
mdl = Model()
set_optimizer(mdl, COSMO.Optimizer)

x = @variable(mdl, x[1:2])
@constraint(mdl, Aa*x .<= ba)
@objective(mdl, Min, 0.5*x'*P*x+q'*x)

status = JuMP.optimize!(mdl)

JuMP.objective_value(mdl)

------------------------------------------------------------------
          COSMO v0.8.1 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2021
------------------------------------------------------------------

Problem:  x ∈ R^{2},
          constraints: A ∈ R^{6x2} (8 nnz),
          matrix size to factor: 8x8,
          Floating-point precision: Float64
Sets:     Nonnegatives) of dim: 6
Settings: ϵ_abs = 1.0e-05, ϵ_rel = 1.0e-05,
          ϵ_prim_inf = 1.0e-04, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 5000,
          scaling iter = 10 (on),
          check termination every 25 iter,
          check infeasibility every 40 iter,
          KKT system solver: QDLDL
Acc:      Anderson Type2{QRDecomp},
          Memory size = 8, RestartedMemory,	
          Safeguarded: true, tol: 2.0
Setup Time: 0.1ms

Iter:	Objective:	Primal Res:	Dual Res:	Rho:
1	-8.4332e-02	6.0000e-01	9.002

-0.12000000000000008

# SCPToolbox

In [63]:
pbm = ConicProgram(solver = COSMO)

x = @new_variable(pbm, 2, "x")

@add_constraint(pbm, NONPOS, x -> Aa*x-ba)

J = @add_cost(pbm, x -> 0.5*x'*P*x+q'*x)

exit_status = solve!(pbm)
flush(stdout)
display(exit_status)
display(objective_value(pbm))

------------------------------------------------------------------
          COSMO v0.8.1 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2021
------------------------------------------------------------------

Problem:  x ∈ R^{2},
          constraints: A ∈ R^{6x2} (8 nnz),
          matrix size to factor: 8x8,
          Floating-point precision: Float64
Sets:     Nonnegatives) of dim: 6
Settings: ϵ_abs = 1.0e-05, ϵ_rel = 1.0e-05,
          ϵ_prim_inf = 1.0e-04, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 5000,
          scaling iter = 10 (on),
          check termination every 25 iter,
          check infeasibility every 40 iter,
          KKT system solver: QDLDL
Acc:      Anderson Type2{QRDecomp},
          Memory size = 8, RestartedMemory,	
          Safeguarded: true, tol: 2.0
Setup Time: 0.07ms

Iter:	Objective:	Primal Res:	Dual Res:	Rho:
1	-8.4332e-02	6.0000e-01	9.00

OPTIMAL::TerminationStatusCode = 1

-0.12000000000000008